In [ ]:
!pip install -qU langchain-openai
!pip install langchain-community
!pip install gradio

In [ ]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API key를 입력하세요.')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history, additional_input_info):
        history_langchain_format = []
        # additional_input_info로 받은 시스템 프롬프트를 랭체인에게 전달할 메시지에 포함시킨다.
        history_langchain_format.append(SystemMessage(content= additional_input_info))
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm(history_langchain_format)
        return gpt_response.content

system_prompt = """
당신은 8세 아동이고, 아이처럼 말하는 것에 능숙하다.
사용자의 질문에 대해 아이처럼 대화를 하여라.
"""

iface = gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="나만의 챗봇과 대화해봐요!!", container=False, scale=5),
        chatbot=gr.Chatbot(height=300), # 채팅창 높이 조절
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        theme="soft",
        examples=[["안뇽"], ["요즘 덥다 ㅠㅠ"], ["점심메뉴 추천바람, 짜장 짬뽕 택 1"]],
        retry_btn="다시 작성하기 ↩",
        undo_btn="이전 대화 삭제 ❌",
        clear_btn="전체 대화 삭제 💫",
        additional_inputs=[
            gr.Textbox(system_prompt, label="System Prompt를 입력해주세요", placeholder="8세 아동이고 아이처럼 말하는 것에 능숙하다.")
        ]
)
iface.launch()